In [ ]:
from pipline.pipline import pipeline_house_data
from model.train_model import train_model
from model.evaluate import evaluate_model
from utils.config import KEYWORDS
import pandas as pd

df = pd.read_csv("data/house_prices.csv")
df_cleaned = pipeline_house_data(df, keywords=KEYWORDS)

model, X_test, y_test = train_model(df_cleaned)
evaluate_model(model, X_test, y_test)